In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/PA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
493,3354717105,PA,"314 S HENDERSON RD SUITE G347, KING OF PRUSSIA...",40.091312,-75.358874,NaN
941,6710297002,PA,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
1347,2197618706,PA,"PO Box 520, BETHEL PARK, PA",40.327570,-80.039498,NaN
4233,7276257209,PA,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5623,3628157101,PA,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6978,7706207108,PA,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7089,5889008400,PA,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",39.959050,-75.151570,NaN
7686,3639927102,PA,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
8677,7421237102,PA,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN
9583,3026777102,PA,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('42')] #PA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/51 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

51


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9510/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
493,3354717105,PA,"314 S HENDERSON RD SUITE G347, KING OF PRUSSIA...",40.091312,-75.358874,[420912058062016]
941,6710297002,PA,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,[]
1347,2197618706,PA,"PO Box 520, BETHEL PARK, PA",40.327570,-80.039498,[420034753013000]
4233,7276257209,PA,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,[]
5623,3628157101,PA,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,[]
6978,7706207108,PA,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,[]
7089,5889008400,PA,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",39.959050,-75.151570,[421010376001013]
7686,3639927102,PA,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,[]
8677,7421237102,PA,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,[]
9583,3026777102,PA,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9510/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
493,3354717105,PA,"314 S HENDERSON RD SUITE G347, KING OF PRUSSIA...",40.091312,-75.358874,420912058062016
941,6710297002,PA,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
1347,2197618706,PA,"PO Box 520, BETHEL PARK, PA",40.327570,-80.039498,420034753013000
4233,7276257209,PA,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5623,3628157101,PA,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6978,7706207108,PA,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7089,5889008400,PA,"417 N 8th St Ste 302 417 N 8th St Ste 302, Phi...",39.959050,-75.151570,421010376001013
7686,3639927102,PA,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
8677,7421237102,PA,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN
9583,3026777102,PA,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
941,6710297002,PA,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
4233,7276257209,PA,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5623,3628157101,PA,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6978,7706207108,PA,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7686,3639927102,PA,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
8677,7421237102,PA,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN
9583,3026777102,PA,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN
10058,4466207104,PA,"208 Flynn Ave., Suite 2A, UNIONTOWN, PA",44.455988,-73.217960,NaN
12639,3687147708,PA,"3821 South Rd, Mukilteo, PA",47.883043,-122.285864,NaN
13283,4892717303,PA,"570 Route 202, North Wales, PA",41.506985,-73.420491,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
493,3354717105,420912058062016
941,6710297002,NaN
1347,2197618706,420034753013000
4233,7276257209,NaN
5623,3628157101,NaN
6978,7706207108,NaN
7089,5889008400,421010376001013
7686,3639927102,NaN
8677,7421237102,NaN
9583,3026777102,NaN


Read in base data

In [15]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/PA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,733439,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14
1,733440,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14
2,733441,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14
3,733442,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14
4,733443,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,733439,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14,NaN
1,733440,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14,NaN
2,733441,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14,NaN
3,733442,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14,NaN
4,733443,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14,NaN
...,...,...,...,...,...,...,...,...
38495,771934,9738208500,PA,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,4.201991e+14,NaN
38496,771935,9754548410,PA,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,4.200343e+14,NaN
38497,771936,9876218305,PA,"514 Poplar St, Columbia, PA",40.037187,-76.502122,4.207101e+14,NaN
38498,771937,9910327007,PA,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,4.207701e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,733439,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,4.200302e+14,NaN,420030203001039.0
1,733440,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,4.204302e+14,NaN,420430223004027.0
2,733441,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,4.200302e+14,NaN,420030203001044.0
3,733442,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,4.209121e+14,NaN,420912059064016.0
4,733443,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,4.200348e+14,NaN,420034838002001.0


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1780617201,PA,"2767 Liberty Avenue, Pittsburgh, PA",40.456180,-79.975512,420030203001039.0
1,1828167306,PA,"3940 Locust Lane, HARRISBURG, PA",40.291012,-76.828894,420430223004027.0
2,2394397105,PA,"2501 Smallman Street, PITTSBURGH, PA",40.455010,-79.980727,420030203001044.0
3,4165028809,PA,"2012 Renaissance Blvd, King of Prussia, PA",40.084581,-75.332199,420912059064016.0
4,4396367001,PA,"285 E Waterfront Dr, HOMESTEAD, PA",40.412345,-79.912886,420034838002001.0
...,...,...,...,...,...,...
38495,9738208500,PA,"686 Glenwood Way, Butler, PA",40.904294,-79.887279,420199108001041.0
38496,9754548410,PA,"968 Perry Hwy 968 Perry Hwy, Pittsburgh, PA",40.537397,-80.035949,420034294002006.0
38497,9876218305,PA,"514 Poplar St, Columbia, PA",40.037187,-76.502122,420710113002019.0
38498,9910327007,PA,"900 Lehigh Valley Mall Dr., WHITEHALL, PA",40.629235,-75.476755,420770057022047.0


In [19]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
941,6710297002,PA,"3001 W Big Beaver Rd, SUMNEYTOWN, PA",42.560203,-83.188765,NaN
4233,7276257209,PA,"528 Billy Sunday Rd, PITTSBURGH, PA",42.002059,-93.603973,NaN
5623,3628157101,PA,"116 Chestnut St. 102, Red Bank, PA",40.347054,-74.073798,NaN
6978,7706207108,PA,"12a Cabot Rd, ALBURTIS, PA",42.407328,-71.076399,NaN
7686,3639927102,PA,"1 Tank Farm and Shippers Rd, box 159, MACUNGIE...",42.100344,-76.001736,NaN
8677,7421237102,PA,"2 Corporation Way Ste 250, READING, PA",42.521600,-70.960160,NaN
9583,3026777102,PA,"20 Indian Dr, Langhorne, PA",41.271711,-72.524810,NaN
10058,4466207104,PA,"208 Flynn Ave., Suite 2A, UNIONTOWN, PA",44.455988,-73.217960,NaN
12639,3687147708,PA,"3821 South Rd, Mukilteo, PA",47.883043,-122.285864,NaN
13283,4892717303,PA,"570 Route 202, North Wales, PA",41.506985,-73.420491,NaN


Convert to File

In [20]:
fips_PA = fips_merge_drop

In [21]:
fips_PA.to_csv("../../../data/state_data/geo/geo_fips/23/PA_fips_scraped.csv") 